### Imports

In [34]:
import zipfile
import os
from tqdm import tqdm

### Load Data

Unzip Data

In [5]:
with zipfile.ZipFile('all_alignments_trimmed.zip', 'r') as zip_ref:
    zip_ref.extractall('Data')

Find FASTA Files

In [12]:
path = './Data/all_alignments_trimmed'
fasta_files = [f for f in os.listdir(path) if f.endswith('.fasta')]
print(fasta_files)

['OG10015_Hmm10-gb.fasta', 'OG10028_Hmm10-gb.fasta', 'OG10087_Hmm10-gb.fasta', 'OG10092_Hmm10-gb.fasta', 'OG10107_Hmm10-gb.fasta', 'OG10131_Hmm10-gb.fasta', 'OG10137_Hmm10-gb.fasta', 'OG10170_Hmm10-gb.fasta', 'OG10206_Hmm10-gb.fasta', 'OG10212_Hmm10-gb.fasta', 'OG10257_Hmm10-gb.fasta', 'OG1026_Hmm10-gb.fasta', 'OG10299_Hmm10-gb.fasta', 'OG10316_Hmm10-gb.fasta', 'OG10325_Hmm10-gb.fasta', 'OG10383_Hmm10-gb.fasta', 'OG10448_Hmm10-gb.fasta', 'OG10449_Hmm10-gb.fasta', 'OG1044_Hmm10-gb.fasta', 'OG10461_Hmm10-gb.fasta', 'OG10511_Hmm10-gb.fasta', 'OG10538_Hmm10-gb.fasta', 'OG10553_Hmm10-gb.fasta', 'OG10561_Hmm10-gb.fasta', 'OG10565_Hmm10-gb.fasta', 'OG10585_Hmm10-gb.fasta', 'OG10619_Hmm10-gb.fasta', 'OG10632_Hmm10-gb.fasta', 'OG10648_Hmm10-gb.fasta', 'OG10672_Hmm10-gb.fasta', 'OG10702_Hmm10-gb.fasta', 'OG10710_Hmm10-gb.fasta', 'OG10769_Hmm10-gb.fasta', 'OG10792_Hmm10-gb.fasta', 'OG10795_Hmm10-gb.fasta', 'OG10803_Hmm10-gb.fasta', 'OG10826_Hmm10-gb.fasta', 'OG10830_Hmm10-gb.fasta', 'OG1085_Hmm10

Function to Get Sequences From FASTA File as Dict

In [ ]:
# returns a dictionary object with sequence names as keys and sequences as values
def get_sequences(fasta_file):
    with open(os.path.join(path, fasta_file), 'r') as file:
        sequences = {}
        current_seq_name = None
        current_sequence = []
        for line in file:
            line = line.strip()
            if line.startswith('>'):  # Start of a new sequence
                if current_seq_name:  # Save the previous sequence
                    sequences[current_seq_name] = ''.join(current_sequence)
                current_seq_name = line[1:]  # Save the sequence name (exclude '>')
                current_sequence = []  # Reset the sequence buffer
            else:
                current_sequence.append(line)  # Collect sequence data
        if current_seq_name:  # Save the last sequence in the file
            sequences[current_seq_name] = ''.join(current_sequence)
    return sequences

Dictionary of Dictionaries Containing Sequences for Every Gene

In [31]:
gene_sequences = {}
for fasta_file in fasta_files:
    sequences = get_sequences(fasta_file)
    gene_name = fasta_file.split('_')[0]
    gene_sequences[gene_name] = sequences

### Calculate Distance Matrix

In [40]:
blosum = {}
with open('blosum62.txt', 'r') as f:
    lines = f.readlines()[6:]
    keys = lines[0].split()
    keys[-1] = '-'
    for i, line in enumerate(lines[1:]):
        blosum[keys[i]] = {k : int(v) for (k,v) in zip(keys, line.split()[1:])}

def calculate_distance(seq1, seq2):
    score = 0
    for i in range(len(seq1)):
        a = '-' if seq1[i] == '?' else seq1[i]
        b = '-' if seq2[i] == '?' else seq2[i]
        score -= blosum[a][b]
    return score

In [42]:
distance_matrices = {}
for gene_name, sequences in tqdm(gene_sequences.items()):
    distance_matrix = {}
    for seq1_name, seq1 in sequences.items():
        distance_matrix[seq1_name] = {}
        for seq2_name, seq2 in sequences.items():
            distance_matrix[seq1_name][seq2_name] = calculate_distance(seq1, seq2)
    distance_matrices[gene_name] = distance_matrix

  2%|▏         | 28/1173 [00:12<08:24,  2.27it/s]


KeyboardInterrupt: 

### Perform Neighbor Joining

In [ ]:
def min_S_value(D, u):
    m = len(D)
    min_S, min_i, min_j = float("inf"),-1,-1
    for k in D:
        for l in D[k]:
            if l!=k:
                crit = (m-2)*D[k][l] - u[k] - u[l]
                if crit < min_S:
                    min_S = crit
                    min_i = k
                    min_j = l
    return (min_i, min_j)

def neighbor_join(D):
    T = {}
    r = len(D)
    new_node = r

    while len(D) > 2:
        print("hello")
        u = {k: sum(D[k].values()) for k in D}
        i, j = min_S_value(D, u)

        # new node combining i and j
        r = new_node
        new_node += 1

        # print(T)
        if i not in T:
            T[i] = {}
        if j not in T:
            T[j] = {}
        T[r] = {}
        # print(T)
        # T[r][i] = 0.5 * (D[i][j] + (u[i] - u[j]) / len(D))
        T[r][i] = 0.5 * (D[i][j] + (u[i] - u[j]) / (len(D) - 2))
        T[i][r] = T[r][i]
        # T[r][j] = 0.5 * (D[i][j] + (u[j] - u[i]) / len(D))
        T[r][j] = 0.5 * (D[i][j] + (u[j] - u[i]) / (len(D) - 2))
        T[j][r] = T[r][j]

        new_dist = {}
        for m in D:
            if m != i and m != j:
                new_dist[m] = 0.5 * (D[i][m] + D[j][m] - D[i][j]) # from lecture
                D[m][r] = new_dist[m]

        D[r] = new_dist

        # remove all of i and j
        del D[i]
        del D[j]
        for k in D:
            if i in D[k]:
                del D[k][i]
            if j in D[k]:
                del D[k][j]

    # print(T)
    extra_shit = list(D.keys())
    i, j = extra_shit[0], extra_shit[1]
    if i not in T:
        T[i] = {}
    if j not in T:
        T[j] = {}
    T[i][j] = D[i][j]
    T[j][i] = D[i][j]

    return T

In [ ]:
tree = neighbor_join(distance_matrices['OG10015'])
tree
print(distance)

hello


IndexError: list index out of range

### Visualize Tree